In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


# Imports

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
os.chdir('/content/drive/MyDrive/python/zs news/')
os.getcwd()

'/content/drive/MyDrive/python/zs news'

# Read Data

In [5]:
train = pd.read_csv('/content/drive/MyDrive/python/zs news/train_file.csv')
test = pd.read_csv('/content/drive/MyDrive/python/zs news/test_file.csv')
print(train.shape, test.shape)

(55932, 11) (37288, 9)


In [6]:
train.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1,-1,-1,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1,-1,-1,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1,-1,-1,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1,-1,-1,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1,-1,-1,0.000000,0.141084


In [7]:
test.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn
0,tFrqIR6Chj,Sliding Economy: FG fights back with N3trn TSA...,With the 2016 budget now passed by the Nationa...,BusinessDay,economy,2016-03-29 01:41:12,0,0,1
1,DVAaGErjlF,Microsoft shows how HoloLens can bring distant...,A recent Microsoft Research video shows how th...,Daily Mail,microsoft,2016-03-29 01:41:27,121,2,13
2,OT9UIZm5M2,"Microsoft’s Twitter Robot Praises Hitler, Trum...",* Microsoft teamed with Bing to create TayTwee...,EURweb,microsoft,2016-03-29 01:47:00,12,1,0
3,lflGp3q2Fj,Flood of Central Bank Moves Can't Get World Ec...,Central bankers have managed to steer the worl...,Bloomberg via Yahoo! Finance,economy,2016-03-29 02:00:00,0,0,3
4,zDYG0SoovZ,USD/JPY: bears lining up on mixed U.S. economy...,"However, this streak of seven-day gains might ...",FXStreet,economy,2016-03-29 02:01:07,3,0,0


# Prepare training data

Creating single dataframe with text and its sentiment

In [8]:
train.duplicated().sum()

0

In [9]:
train.columns

Index(['IDLink', 'Title', 'Headline', 'Source', 'Topic', 'PublishDate',
       'Facebook', 'GooglePlus', 'LinkedIn', 'SentimentTitle',
       'SentimentHeadline'],
      dtype='object')

In [10]:
df1 = train[['Title','SentimentTitle']].copy()
df1.columns = ['text','sentiment']
df1.shape

(55932, 2)

In [11]:
df2 = train[['Headline','SentimentHeadline']].copy()
df2.columns = ['text','sentiment']
df2.shape

(55932, 2)

In [12]:
df = pd.concat([df1,df2], axis=0, ignore_index=True)
df.shape

(111864, 2)

In [13]:
df.head()

,text,sentiment
0,Obama Lays Wreath at Arlington National Cemetery,0.000000
1,A Look at the Health of the Chinese Economy,0.208333
2,Nouriel Roubini: Global Economy Not Back to 2008,-0.425210
3,Finland GDP Expands In Q4,0.000000
4,"Tourism, govt spending buoys Thai economy in J...",0.000000


# Cleaning df

In [14]:
print(df.shape)
df = df.drop_duplicates().reset_index(drop=True).copy()
df = df.dropna(how='any')
print(df.shape)

(111864, 2)
(101075, 2)


In [15]:
df.head()

,text,sentiment
0,Obama Lays Wreath at Arlington National Cemetery,0.000000
1,A Look at the Health of the Chinese Economy,0.208333
2,Nouriel Roubini: Global Economy Not Back to 2008,-0.425210
3,Finland GDP Expands In Q4,0.000000
4,"Tourism, govt spending buoys Thai economy in J...",0.000000


# Creating X and y

In [16]:
X=df['text'].values.reshape(-1,1)
y=df['sentiment'].values.reshape(-1,1)
print(X.shape,y.shape)

(101075, 1) (101075, 1)


In [17]:
X[:10]

array([['Obama Lays Wreath at Arlington National Cemetery'],
       ['A Look at the Health of the Chinese Economy'],
       ['Nouriel Roubini: Global Economy Not Back to 2008'],
       ['Finland GDP Expands In Q4'],
       ['Tourism, govt spending buoys Thai economy in January'],
       ['Intellitec Solutions to Host 13th Annual Spring Microsoft Dynamics User Group'],
       ['Monday, 29 Feb 2016'],
       ['Obama, stars pay a musical tribute to Ray Charles'],
       ['Fire claims more than 100-year-old barn in Hancock County'],
       ["Microsoft's new Windows 10 ad targets Apple"]], dtype=object)

In [18]:
y[:10]

array([[ 0.        ],
       [ 0.20833333],
       [-0.42521003],
       [ 0.        ],
       [ 0.        ],
       [-0.07537784],
       [ 0.        ],
       [ 0.08333333],
       [-0.17392527],
       [-0.05953621]])

# Text Cleaning

In [19]:
import re
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm

In [ ]:
# "not" in STOP_WORDS

True

In [20]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
punct = "!#$%&()*+,-./:;<=>?@[\]^_`{|}~"
print(punct)

!#$%&()*+,-./:;<=>?@[\]^_`{|}~


In [22]:
# loading spacy english model
nlp = spacy.load('en_core_web_sm')

In [23]:
doc = nlp("Id cant do this and it's difficult. My website n't www.google.com")
for token in doc:
  print(token.lemma_)

-PRON-
would
can
not
do
this
and
-PRON-
be
difficult
.
-PRON-
website
n't
www.google.com


In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [25]:
# text cleaning function

def text_cleaning(X):
  corpus = []
  for i in tqdm(range(0,len(X))):
    sen = re.sub("http?\S+"," ",X[i][0])
    sen = re.sub('[^a-zA-Z]', ' ', sen)
    doc = nlp(sen)
    tokens = []
    for token in doc:
      if token.lemma_ != "-PRON-":
        temp = token.lemma_.lower().strip()
      else:
        temp = token.lower_.strip()
      tokens.append(temp)
      tokens = [token for token in tokens if token not in punct]
    corpus.append(' '.join(tokens))
  return corpus

In [26]:
corpus = text_cleaning(X)
print('cleaning done..')

100%|██████████| 101075/101075 [17:42<00:00, 95.14it/s]

cleaning done..


In [27]:
len(corpus)

101075

In [ ]:
# sanity check for corpus

In [28]:
test.columns

Index(['IDLink', 'Title', 'Headline', 'Source', 'Topic', 'PublishDate',
       'Facebook', 'GooglePlus', 'LinkedIn'],
      dtype='object')

In [29]:
X_test_title= test['Title'].values.reshape(-1,1).copy()
print(X_test_title.shape)

X_test_head= test['Headline'].values.reshape(-1,1).copy()
print(X_test_head.shape)

(37288, 1)
(37288, 1)


In [30]:
corpus_test_title = text_cleaning(X_test_title)
print('cleaning done..')

corpus_test_head = text_cleaning(X_test_head)
print('cleaning done..')

100%|██████████| 37288/37288 [05:35<00:00, 111.25it/s]


cleaning done..


100%|██████████| 37288/37288 [07:10<00:00, 86.59it/s]

cleaning done..


# Find max len

In [31]:
max([len(i[0].split()) for i in X])

80

In [32]:
max([len(i.split()) for i in corpus])

87

In [33]:
max([len(i.split()) for i in corpus_test_title])

23

In [34]:
max([len(i.split()) for i in corpus_test_head])

82

# Onehot

In [35]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
voc_size = 25000

In [36]:
print(len(corpus),len(corpus_test_title),len(corpus_test_head))

101075 37288 37288


In [37]:
corpus_tot = corpus+corpus_test_title+corpus_test_head
len(corpus_tot)

175651

In [38]:
onehot_rep = [one_hot(words, n=voc_size) for words in corpus_tot]

In [39]:
onehot_rep[:5]

[[10507, 6227, 5171, 12021, 24827, 19580, 10342],
 [16457, 9869, 12021, 15517, 15069, 14232, 15517, 5543, 4990],
 [13338, 3478, 14500, 4990, 6992, 19737, 1679],
 [21014, 17407, 21072, 9974, 16984],
 [7604, 15981, 23612, 1171, 7988, 4990, 9974, 21371]]

In [40]:
len(onehot_rep)

175651

In [41]:
sent_length = 90
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_docs[0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0 10507
  6227  5171 12021 24827 19580 10342]


In [42]:
embedded_docs_train = embedded_docs[:len(corpus)]
embedded_docs_test_title = embedded_docs[len(corpus):len(corpus)+len(corpus_test_title)]
embedded_docs_test_head = embedded_docs[len(corpus)+len(corpus_test_title):]

print(len(corpus),len(corpus_test_title),len(corpus_test_head))
print(len(embedded_docs_train),len(embedded_docs_test_title),len(embedded_docs_test_head))

101075 37288 37288
101075 37288 37288


# Split data

In [43]:
embedded_docs_train.shape, y.shape, embedded_docs_test_title.shape, embedded_docs_test_head.shape

((101075, 90), (101075, 1), (37288, 90), (37288, 90))

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(embedded_docs_train, y, test_size=0.33, random_state = 42, shuffle=True)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((67720, 90), (67720, 1), (33355, 90), (33355, 1))

# Model

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Embedding, Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [46]:
embedding_dim=128


model_bilstm=Sequential()
model_bilstm.add(Embedding(voc_size,embedding_dim,input_length=sent_length))
model_bilstm.add(Bidirectional(LSTM(64)))
model_bilstm.add(Dropout(0.25))
model_bilstm.add(Flatten())
model_bilstm.add(Dense(1, activation='tanh'))
model_bilstm.compile(loss='mean_squared_error',optimizer='adam')
print(model_bilstm.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 90, 128)           3200000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 3,298,945
Trainable params: 3,298,945
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# from keras.utils.vis_utils import plot_model

# plot_model(model=model_bilstm, to_file='model.png')

In [47]:
early_s = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

In [48]:
history = model_bilstm.fit(X_train, y_train, validation_data=(X_val,y_val), 
                           batch_size=512, epochs=20, callbacks=[early_s], verbose=1)

Epoch 1/20
133/133 [==============================] - 18s 73ms/step - loss: 0.0122 - val_loss: 0.0091
Epoch 2/20
133/133 [==============================] - 9s 66ms/step - loss: 0.0081 - val_loss: 0.0088
Epoch 3/20
133/133 [==============================] - 9s 66ms/step - loss: 0.0072 - val_loss: 0.0087
Epoch 4/20
133/133 [==============================] - 9s 66ms/step - loss: 0.0064 - val_loss: 0.0085
Epoch 5/20
133/133 [==============================] - 9s 66ms/step - loss: 0.0056 - val_loss: 0.0083
Epoch 6/20
133/133 [==============================] - 9s 66ms/step - loss: 0.0049 - val_loss: 0.0081
Epoch 7/20
133/133 [==============================] - 9s 66ms/step - loss: 0.0044 - val_loss: 0.0083
Epoch 8/20
133/133 [==============================] - 9s 66ms/step - loss: 0.0040 - val_loss: 0.0084
Epoch 9/20
133/133 [==============================] - 9s 67ms/step - loss: 0.0037 - val_loss: 0.0086
Epoch 10/20
133/133 [==============================] - 9s 66ms/step - loss: 0.0034 - val_l

# Prediction

In [49]:
y_pred_title = model_bilstm.predict(embedded_docs_test_title)
y_pred_head = model_bilstm.predict(embedded_docs_test_head)

In [50]:
y_pred_title

array([[ 0.02532711],
       [-0.09990197],
       [-0.05235893],
       ...,
       [ 0.4050634 ],
       [ 0.09002601],
       [-0.10041559]], dtype=float32)

In [51]:
y_pred_head

array([[-0.08304027],
       [ 0.11378475],
       [-0.13867153],
       ...,
       [-0.19634722],
       [ 0.085851  ],
       [-0.02667028]], dtype=float32)

# csv file

In [52]:
y_pred_title.reshape(1,-1)[0]

array([ 0.02532711, -0.09990197, -0.05235893, ...,  0.4050634 ,
        0.09002601, -0.10041559], dtype=float32)

In [53]:
sub = pd.DataFrame({'IDLink': test['IDLink'],
                    'SentimentTitle': y_pred_title.reshape(1,-1)[0],
                    'SentimentHeadline': y_pred_head.reshape(1,-1)[0]})
print(sub.shape, test.shape)

(37288, 3) (37288, 9)


In [54]:
sub.head(10)

,IDLink,SentimentTitle,SentimentHeadline
0,tFrqIR6Chj,0.025327,-0.083040
1,DVAaGErjlF,-0.099902,0.113785
2,OT9UIZm5M2,-0.052359,-0.138672
3,lflGp3q2Fj,-0.118598,-0.190379
4,zDYG0SoovZ,-0.070764,0.070364
5,xwr9uOYgEj,0.170573,0.002234
6,X8BqN4I4xZ,-0.047358,0.055217
7,ESXBINkNt4,0.144588,0.199827
8,uUjZusP8FU,-0.078118,0.018881
9,VIxYwukhvK,0.018546,-0.004881


In [55]:
sub.to_csv('/content/drive/MyDrive/python/zs news/sub_file2.csv', index=False)